In [1]:
import json
import logging
import os
import sys
from typing import NoReturn

from pydantic import ValidationError

from core import ExcelHandler, NetworkSimulation, PipSimInput

logger = logging.getLogger("Hydraulics")

In [2]:
from wave_hydraulics import *

In [3]:
config = load_config("inputs.json")

INFO:Hydraulics:Input file loaded successfully.


In [4]:
pipsim_files = [
    file for file in os.listdir(config.FOLDER_DIRECTORY) if file.endswith(".pips")
]
pipsim_files.remove(str(config.MODEL_FILENAME))

In [5]:
# pipsim_files = pipsim_files[:1]
pipsim_files = ["Overall_Min_S-LP-EO_BAB HP.pips", "Overall_Min_S-LP-LO_BAB HP.pips"]
pipsim_files

['Overall_Min_S-LP-EO_BAB HP.pips', 'Overall_Min_S-LP-LO_BAB HP.pips']

In [6]:
for model_filename in pipsim_files:
    ns = NetworkSimulation(config.FOLDER_DIRECTORY, model_filename, config.EXCEL_FILE)
    ns.initialize_excel_handler(config.PIPSIM_INPUT_SHEET, config.CONDITIONS_SHEET)
    ns.run_existing_model(source_name=config.SOURCE_NAME, pump_name=config.PUMP_NAME)

INFO:NetworkSimulation:------------Network Simulation Object Created----------------
Model Path: c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\Overall_Min_S-LP-EO_BAB HP.pips

INFO:NetworkSimulation:Model prepared
 Case:Overall_Min
 Condition: S-LP-EO
INFO:manta.server.manager:Starting PIPESIM server on thread ID: 11300
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:NetworkSimulation:Getting boundary conditions.....
INFO:NetworkSimulation:Set Global Conditions
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Sim